In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt 

# 데이터 불러오기

In [2]:
# 날씨데이터
df_weather = pd.read_csv('weather_treatment.csv')

# 관중데이터
df_spec = pd.read_csv('spectator.csv')

# KBO 경기결과 데이터
df_kbo = pd.read_csv('result_kbo.csv')

### 날씨-관중 데이터 합치기

> 날씨데이터 전처리

In [3]:
# 날짜데이터 - 하이픈 삭제

df_weather['일시'] = df_weather['일시'].str.replace('-', '')
df_weather


,Unnamed: 0,지점,지점명,일시,평균기온,최저기온,최고기온,일강수량,풍속,습도,강수여부
0,0,108,서울,20210101,-4.2,-9.8,1.6,0.0,2.0,64.0,sunny
1,1,108,서울,20210102,-5.0,-8.4,-1.4,0.0,2.6,38.5,sunny
2,2,108,서울,20210103,-5.6,-9.1,-2.0,0.0,2.0,45.0,sunny
3,3,108,서울,20210104,-3.5,-8.4,0.3,0.0,1.7,51.4,sunny
4,4,108,서울,20210105,-5.5,-9.9,-2.1,0.0,2.9,52.8,sunny
...,...,...,...,...,...,...,...,...,...,...,...
11815,11815,159,부산,20240323,16.1,12.3,20.5,0.7,5.2,73.6,rainy
11816,11816,159,부산,20240324,11.3,10.1,13.6,3.7,4.2,77.5,rainy
11817,11817,159,부산,20240325,10.5,8.7,12.4,9.4,4.2,82.4,rainy
11818,11818,159,부산,20240326,8.8,6.2,12.9,9.8,3.8,75.4,rainy


In [5]:

# 원하는 정보만 남기기

cond = ['지점명', '일시', '평균기온', '일강수량', '풍속', '습도', '강수여부']
df_weather2 = df_weather[cond]
df_weather2

,지점명,일시,평균기온,일강수량,풍속,습도,강수여부
0,서울,20210101,-4.2,0.0,2.0,64.0,sunny
1,서울,20210102,-5.0,0.0,2.6,38.5,sunny
2,서울,20210103,-5.6,0.0,2.0,45.0,sunny
3,서울,20210104,-3.5,0.0,1.7,51.4,sunny
4,서울,20210105,-5.5,0.0,2.9,52.8,sunny
...,...,...,...,...,...,...,...
11815,부산,20240323,16.1,0.7,5.2,73.6,rainy
11816,부산,20240324,11.3,3.7,4.2,77.5,rainy
11817,부산,20240325,10.5,9.4,4.2,82.4,rainy
11818,부산,20240326,8.8,9.8,3.8,75.4,rainy


> 관중데이터 전처리

In [11]:
# 년월일 두자리수 맞추고 str형태로 변경

df_spec['월'] = df_spec['월'].apply(lambda x : str(x).zfill(2))
df_spec['일'] = df_spec['일'].apply(lambda x : str(x).zfill(2))
df_spec['연도'] = df_spec['연도'].astype(str)

# 일시 컬럼 생성

df_spec['일시'] = df_spec['연도'] + df_spec['월'] + df_spec['일']
df_spec

# 홈팀 원정팀 (기아/SSG 팀명 변경)
df_spec['원정팀'] = df_spec['원정팀'].str.replace("KIA", "기아", 1)
df_spec['홈팀'] = df_spec['홈팀'].str.replace("KIA", "기아", 1)
df_spec['원정팀'] = df_spec['원정팀'].str.replace("SSG", "SK", 1)
df_spec['홈팀'] = df_spec['홈팀'].str.replace("SSG", "SK", 1)
df_spec['원정팀'] = df_spec['원정팀'].str.replace("키움히어로즈", "키움", 1)
df_spec['홈팀'] = df_spec['홈팀'].str.replace("키움히어로즈", "키움", 1)
df_spec



# 경기코드 생성
def gamecode(data) : 
        data['code'] = data['일시'] + data['원정팀'].str[:1] + data['홈팀'].str[:1]

gamecode(df_spec)
df_spec


# 지점명 생성
df_spec['지점명'] = df_spec['경기장'].str[:2]
df_spec

,연도,월,일,요일,홈팀,원정팀,경기장,날씨,평균기온,관중수,일시,code,지점명
0,2023,10,17,화,기아,NC,광주-기아 챔피언스 필드,맑음,21.7,"10,175",20231017,20231017N기,광주
1,2023,10,17,화,SK,두산,인천SSG 랜더스필드,맑음,19.5,"21,007",20231017,20231017두S,인천
2,2023,10,16,월,두산,SK,서울 잠실 야구장,맑음,19.6,"15,850",20231016,20231016S두,서울
3,2023,10,16,월,한화,롯데,대전 한화생명 이글스파크,맑음,20.7,"12,000",20231016,20231016롯한,대전
4,2023,10,16,월,기아,NC,광주-기아 챔피언스 필드,맑음,20.6,"5,251",20231016,20231016N기,광주
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,2021,04,03,토,두산,기아,서울 잠실 야구장,비,18.0,0,20210403,20210403기두,서울
2434,2021,04,03,토,키움,삼성,서울고척스카이돔,비,18.0,1665,20210403,20210403삼키,서울
2435,2021,04,03,토,KT,한화,수원케이티위즈파크,비,18.6,0,20210403,20210403한K,수원
2436,2021,04,03,토,NC,LG,창원NC파크,비,17.0,0,20210403,20210403LN,창원


In [12]:

# 원하는 데이터만 남기기

cond2 = ['일시', '요일', '홈팀', '원정팀', '경기장', '지점명', '관중수', 'code']
df_spec2 = df_spec[cond2]
df_spec2

,일시,요일,홈팀,원정팀,경기장,지점명,관중수,code
0,20231017,화,기아,NC,광주-기아 챔피언스 필드,광주,"10,175",20231017N기
1,20231017,화,SK,두산,인천SSG 랜더스필드,인천,"21,007",20231017두S
2,20231016,월,두산,SK,서울 잠실 야구장,서울,"15,850",20231016S두
3,20231016,월,한화,롯데,대전 한화생명 이글스파크,대전,"12,000",20231016롯한
4,20231016,월,기아,NC,광주-기아 챔피언스 필드,광주,"5,251",20231016N기
...,...,...,...,...,...,...,...,...
2433,20210403,토,두산,기아,서울 잠실 야구장,서울,0,20210403기두
2434,20210403,토,키움,삼성,서울고척스카이돔,서울,1665,20210403삼키
2435,20210403,토,KT,한화,수원케이티위즈파크,수원,0,20210403한K
2436,20210403,토,NC,LG,창원NC파크,창원,0,20210403LN


#### 날씨-관중 데이터 병합하기

In [27]:
# 날씨데이터  df_weather2
# 관중데이터  df_spec2
# 관중데이터로 left join / on = 일시

weather_spec = pd.merge(left = df_weather2, right = df_spec2, how = 'right', on = ['일시', '지점명'])

# 일시 기준으로 내림차순

weather_spec = spec_weather.sort_values(by=['일시'], ascending= True)
weather_spec.reset_index(drop = True, inplace = True)
weather_spec

,지점명,일시,평균기온,일강수량,풍속,습도,강수여부,요일,홈팀,원정팀,경기장,관중수,code
0,인천,20210403,14.1,54.7,3.1,79.6,rainy,토,SK,롯데,인천SSG 랜더스필드,0,20210403롯S
1,서울,20210403,14.9,56.2,3.1,80.8,rainy,토,두산,기아,서울 잠실 야구장,0,20210403기두
2,창원,20210403,15.4,38.7,1.0,84.8,rainy,토,NC,LG,창원NC파크,0,20210403LN
3,수원,20210403,15.4,46.7,1.9,85.4,rainy,토,KT,한화,수원케이티위즈파크,0,20210403한K
4,서울,20210403,14.9,56.2,3.1,80.8,rainy,토,키움,삼성,서울고척스카이돔,1665,20210403삼키
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,대전,20231016,14.7,0.0,1.7,64.5,sunny,월,한화,롯데,대전 한화생명 이글스파크,"12,000",20231016롯한
2434,광주,20231016,15.8,0.0,0.9,77.8,sunny,월,기아,NC,광주-기아 챔피언스 필드,"5,251",20231016N기
2435,서울,20231016,14.7,0.0,2.4,57.0,sunny,월,두산,SK,서울 잠실 야구장,"15,850",20231016S두
2436,인천,20231017,14.8,0.0,1.6,50.8,sunny,화,SK,두산,인천SSG 랜더스필드,"21,007",20231017두S


## 관중-날씨데이터 & 승리 데이터 합치기

In [26]:
df_kbo

# 필요한 컬럼만 추출하기
cond3 = ['code', 'A', 'B', '승리팀']
df_kbo2 = df_kbo[cond3]
df_kbo2

,code,A,B,승리팀
0,20210403삼키,1.0,6.0,키움
1,20210404기두,1.0,4.0,두산
2,20210404롯S,3.0,5.0,SK
3,20210404LN,2.0,1.0,LG
4,20210404한K,2.0,3.0,KT
...,...,...,...,...
1789,20230923두L,4.0,12.0,LG
1790,20230924K롯,4.0,8.0,롯데
1791,20230924한삼,2.0,6.0,삼성
1792,20230924SN,2.0,6.0,NC


In [35]:
# 왼쪽 weather_spec
# 오른쪽 df_kbo2 

eda_data = pd.merge(left = weather_spec, right = df_kbo2, on = 'code')
eda_data

cond_final = ['code', '일시', '요일', '평균기온', '일강수량', '풍속', '습도', '강수여부', '경기장', '지점명', '원정팀', '홈팀', 'A', 'B', '승리팀']
final_eda_data = eda_data[cond_final]
final_eda_data

,code,일시,요일,평균기온,일강수량,풍속,습도,강수여부,경기장,지점명,원정팀,홈팀,A,B,승리팀
0,20210403삼키,20210403,토,14.9,56.2,3.1,80.8,rainy,서울고척스카이돔,서울,삼성,키움,1.0,6.0,키움
1,20210404LN,20210404,일,11.3,14.0,2.6,77.0,rainy,창원NC파크,창원,LG,NC,2.0,1.0,LG
2,20210404롯S,20210404,일,11.4,0.5,4.4,64.8,rainy,인천SSG 랜더스필드,인천,롯데,SK,3.0,5.0,SK
3,20210404기두,20210404,일,11.9,0.6,3.2,65.8,rainy,서울 잠실 야구장,서울,기아,두산,1.0,4.0,두산
4,20210404삼키,20210404,일,11.9,0.6,3.2,65.8,rainy,서울고척스카이돔,서울,삼성,키움,4.0,7.0,키움
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,20230913KN,20230913,수,26.3,0.0,0.8,76.8,sunny,창원NC파크,창원,KT,NC,8.0,2.0,KT
719,20230915L한,20230915,금,21.9,15.1,0.8,93.5,rainy,대전 한화생명 이글스파크,대전,LG,한화,6.0,15.0,한화
720,20230917두기,20230917,일,24.7,3.2,0.7,95.4,rainy,광주-기아 챔피언스 필드,광주,두산,기아,3.0,1.0,두산
721,20230920LK,20230920,수,20.7,36.6,1.8,89.8,rainy,수원케이티위즈파크,수원,LG,KT,4.0,0.0,LG
